## fine-tuning

In [1]:
'''
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train3/
        dog/
            dog001.jpg
            dog002.jpg
            ...
        cat/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dog/
            dog001.jpg
            dog002.jpg
            ...
        cat/
            cat001.jpg
            cat002.jpg
            ...
```
'''
from PIL import Image

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.callbacks import *
from keras.optimizers import *
from keras.utils import *

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 299, 299

train_data_dir = 'data/train3'
valid_data_dir = 'data/validation'

nb_train_samples = 19944
nb_validation_samples = 4986
#batch_size = 277  #19944/277=72  4986/277=18
#batch_size = 72   #19944/72=277  4986/72=69.25
batch_size = 16   #19944/72=277  4986/72=69.25

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
# build the VGG16 network

#构造模型
x_input = Input((299, 299, 3))
x_input = Lambda(xception.preprocess_input)(x_input)

base_model = Xception(input_tensor=x_input, weights='imagenet', include_top=False, pooling = 'avg')

for layer in base_model.layers:
    layer.trainable = False
    
x = Dropout(0.5)(base_model.output)
x = Dense(1, activation='sigmoid',kernel_regularizer=regularizers.l2(0.001))(x)
model = Model(base_model.input, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

gen = ImageDataGenerator(rotation_range=90,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True)
val_gen = ImageDataGenerator()
train_generator = gen.flow_from_directory(train_data_dir, (299, 299), shuffle=True, 
                                          batch_size=64,class_mode='binary')
valid_generator = val_gen.flow_from_directory(valid_data_dir, (299, 299), shuffle=True, 
                                          batch_size=32,class_mode='binary')

for i in range(len(model.layers)):
    print(i,model.layers[i].name)


Found 19944 images belonging to 2 classes.
Found 4986 images belonging to 2 classes.
0 input_1
1 lambda_1
2 block1_conv1
3 block1_conv1_bn
4 block1_conv1_act
5 block1_conv2
6 block1_conv2_bn
7 block1_conv2_act
8 block2_sepconv1
9 block2_sepconv1_bn
10 block2_sepconv2_act
11 block2_sepconv2
12 block2_sepconv2_bn
13 conv2d_1
14 block2_pool
15 batch_normalization_1
16 add_1
17 block3_sepconv1_act
18 block3_sepconv1
19 block3_sepconv1_bn
20 block3_sepconv2_act
21 block3_sepconv2
22 block3_sepconv2_bn
23 conv2d_2
24 block3_pool
25 batch_normalization_2
26 add_2
27 block4_sepconv1_act
28 block4_sepconv1
29 block4_sepconv1_bn
30 block4_sepconv2_act
31 block4_sepconv2
32 block4_sepconv2_bn
33 conv2d_3
34 block4_pool
35 batch_normalization_3
36 add_3
37 block5_sepconv1_act
38 block5_sepconv1
39 block5_sepconv1_bn
40 block5_sepconv2_act
41 block5_sepconv2
42 block5_sepconv2_bn
43 block5_sepconv3_act
44 block5_sepconv3
45 block5_sepconv3_bn
46 add_4
47 block6_sepconv1_act
48 block6_sepconv1
49 bl

In [4]:
#训练模型并保存在验证集上损失函数最小的权重
filepath="xception-best_weight_freeze.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples//batch_size,
        epochs=5,
        validation_data=valid_generator,
        validation_steps=nb_validation_samples//batch_size,
        callbacks = callbacks_list)


Epoch 1/5


AbortedError: Operation received an exception:Status: 3, message: could not set native handle, in file tensorflow/core/kernels/mkl_conv_ops.cc:652
	 [[Node: block1_conv1/convolution = _MklConv2D[T=DT_FLOAT, _kernel="MklOp", data_format="NHWC", padding="VALID", strides=[1, 2, 2, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](lambda_1/sub, block1_conv1/kernel/read, DMT/_0, DMT/_1)]]

Caused by op 'block1_conv1/convolution', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py"

In [ ]:
model.summary()

In [ ]:
predict_on_xception(12500, 299, 299, test_data_dir, model, "xception-best_weight_freeze.h5", "pred-xception-freeze.csv")

In [ ]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')



In [ ]:
# fine-tune the model
#model.fit_generator(
#    train_generator,
#    samples_per_epoch=nb_train_samples,
#    epochs=epochs,
#    validation_data=validation_generator,
#    nb_val_samples=nb_validation_samples)

# fine-tune the model
epochs = 10

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples//batch_size)

In [ ]:
from IPython.display import display
import matplotlib.pyplot as plt

X_val_sample, _ = next(validation_generator)
#y_pred = model.predict(X_val_sample)

y_pred = base_model.predict(X_val_sample)



In [ ]:
print (len(X_val_sample))
print (y_pred.shape)


In [ ]:
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator, array_to_img

nb_sample = 10

for x, y in zip(X_val_sample[:nb_sample], y_pred.flatten()[:nb_sample]):
    s = pd.Series({'Cat': 1-y, 'Dog': y})
    axes = s.plot(kind='bar')
    axes.set_xlabel('Class')
    axes.set_ylabel('Probability')
    axes.set_ylim([0, 1])
    plt.show()

    img = array_to_img(x)
    display(img)